## WELCOME TO WEEK 6

The Epic Finale Week

And

# WELCOME TO THE **M**ODEL **C**ONTEXT **P**ROTOCOL!

And welcome back to OpenAI Agents SDK ❤️❤️❤️

### Please note

There may be changes here from the video as I'm always making **updates**!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">To my Windows PC people - an important announcement **WSL**</h2>
            <span style="color:#ff7800;">I have unpleasant news. There's a problem running MCP Servers on Windows PCs; Mac and Linux is fine. This is a known issue as of May 4th, 2025. I asked o3 with Deep Research to try to find workarounds; it <a href="https://chatgpt.com/share/6817bbc3-3d0c-8012-9b51-631842470628">confirmed the issue</a> and confirmed the workaround.<br/><br/>
            The workaround is a bit of a bore. It is to take advantage of "WSL", the Microsoft approach for running Linux on your PC. You'll need to carry out more setup instructions! But it's quick, and several students have confirmed that this works perfectly for them, then the Week 6 MCP labs work. Plus, WSL is actually a great way to build software on your Windows PC.<br/>
            The WSL Setup instructions are in the Setup folder, <a href="../setup/SETUP-WSL.md">in the file called SETUP-WSL.md here</a>. I do hope this only holds you up briefly - you should be back up and running quickly. Oh the joys of working with bleeding-edge technology!<br/><br/>
            With many thanks to students Markus, Abhi, Hui-Ling, and several others, for helping me work on it and confirming the fix.
            </span>
        </td>
    </tr>
</table>

In [1]:
# The imports

from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os

In [2]:
load_dotenv(override=True)

True

### Let's use MCP in OpenAI Agents SDK

1. Create a Client

2. Have it spawn a server

3. Collect the tools that the server can use

Let's try the Fetch mcp-server that we looked at last week

In [3]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=30) as server:
    fetch_tools = await server.list_tools()

# fetch_tools
print (f"Tools length: {len(fetch_tools)}\n")
for tool in fetch_tools:
    print (f"Tool name: '{tool.name}', Description: '{tool.description}'")

Tools length: 1

Tool name: 'fetch', Description: 'Fetches a URL from the internet and optionally extracts its contents as markdown.

Although originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.'


## Extra installation step - if you don't have "node" on your computer

The next MCP tool uses node (the Javascript Server), and it needs you to have the command 'npx' installed on your computer.

You may already have this, but if not, here are super clear instructions on exactly what to do, courtesy of our friend.  
And thank you to student avid_learner for pointing this out.

https://chatgpt.com/share/68103af2-e2dc-8012-b259-bc135a23273b


### And now repeat for 2 more!

In [4]:
playwright_params = {"command": "npx","args": [ "@playwright/mcp@latest"]}

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=30) as server:
    playwright_tools = await server.list_tools()

# playwright_tools
print (f"Tools length: {len(playwright_tools)}\n")
for tool in playwright_tools:
    print (f"Tool name: '{tool.name}', Description: '{tool.description}'")

Tools length: 25

Tool name: 'browser_close', Description: 'Close the page'
Tool name: 'browser_resize', Description: 'Resize the browser window'
Tool name: 'browser_console_messages', Description: 'Returns all console messages'
Tool name: 'browser_handle_dialog', Description: 'Handle a dialog'
Tool name: 'browser_file_upload', Description: 'Upload one or multiple files'
Tool name: 'browser_install', Description: 'Install the browser specified in the config. Call this if you get an error about the browser not being installed.'
Tool name: 'browser_press_key', Description: 'Press a key on the keyboard'
Tool name: 'browser_navigate', Description: 'Navigate to a URL'
Tool name: 'browser_navigate_back', Description: 'Go back to the previous page'
Tool name: 'browser_navigate_forward', Description: 'Go forward to the next page'
Tool name: 'browser_network_requests', Description: 'Returns all network requests since loading the page'
Tool name: 'browser_pdf_save', Description: 'Save page as PD

In [5]:

sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=30) as server:
    file_tools = await server.list_tools()

# file_tools
print (f"Tools length: {len(file_tools)}\n")
for tool in file_tools:
    print (f"Tool name: '{tool.name}', Description: '{tool.description[:50]}'")

Tools length: 11

Tool name: 'read_file', Description: 'Read the complete contents of a file from the file'
Tool name: 'read_multiple_files', Description: 'Read the contents of multiple files simultaneously'
Tool name: 'write_file', Description: 'Create a new file or completely overwrite an exist'
Tool name: 'edit_file', Description: 'Make line-based edits to a text file. Each edit re'
Tool name: 'create_directory', Description: 'Create a new directory or ensure a directory exist'
Tool name: 'list_directory', Description: 'Get a detailed listing of all files and directorie'
Tool name: 'directory_tree', Description: 'Get a recursive tree view of files and directories'
Tool name: 'move_file', Description: 'Move or rename files and directories. Can move fil'
Tool name: 'search_files', Description: 'Recursively search for files and directories match'
Tool name: 'get_file_info', Description: 'Retrieve detailed metadata about a file or directo'
Tool name: 'list_allowed_directories', Descript

### And now.. bring on the Agent with Tools!

In [9]:
instructions = """
Use internet search tools to find the information you need—do not open a browser or manually click links.
Leverage the search API to access multiple sources, read summaries, and extract key details.
If one result isn't helpful, try different queries or results.
Continue searching and refining until you fully complete the assignment.
"""

food_menu = "Banoffee Pie"
message = f"Find a great recipe for {food_menu}, then summarize it in markdown to {food_menu.lower().replace(" ", "_")}-recipe.md in the sandbox directory."

async with MCPServerStdio(params=files_params, client_session_timeout_seconds=30) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=30) as mcp_server_browser:
        agent = Agent(
            name="investigator", 
            instructions=instructions, 
            model="gpt-4.1-mini",
            mcp_servers=[mcp_server_files, mcp_server_browser]
            )
        with trace("investigate"):
            result = await Runner.run(agent, message)
            print(result.final_output)


I have found a great recipe for Banoffee Pie and summarized it in markdown format. The summary is saved in the file sandbox/banoffee_pie-recipe.md. If you want, I can show you the content or assist you with anything else related to the recipe.


### Check out the trace

https://platform.openai.com/traces

### Now take a look at some MCP marketplaces

https://mcp.so

https://glama.ai/mcp

https://smithery.ai/

https://huggingface.co/blog/LLMhacker/top-11-essential-mcp-libraries

HuggingFace great community article:
https://huggingface.co/blog/Kseniase/mcp



